In [89]:
pip install imblearn

     |████████████████████████████████| 167 kB 4.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### I. The Green dataset

In [1]:
import pandas as pd
url1="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_74d43ea5c0b140d48e018500c2cbf168.csv"
df1=pd.read_csv(url1)
df1.head()

,cervix_area,os_area,walls_area,speculum_area,artifacts_area,cervix_artifacts_area,os_artifacts_area,walls_artifacts_area,speculum_artifacts_area,cervix_specularities_area,...,fit_ellipse_goodness,dist_to_center_cervix,dist_to_center_os,experts::0,experts::1,experts::2,experts::3,experts::4,experts::5,consensus
0,0.344647,0.003080,0.047522,0.288216,0.178585,0.016564,0.0,0.043500,0.010149,0.000133,...,85.474311,0.265933,0.346294,1.0,0.0,0.0,1.0,1.0,1.0,1.0
1,0.165329,0.000000,0.048236,0.504736,0.502783,0.007012,0.0,0.097405,0.973837,0.004055,...,124.794129,1.000000,0.283059,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.457010,0.001681,0.242888,0.212859,0.000000,0.000000,0.0,0.000000,0.000000,0.001756,...,94.948697,0.518740,0.419375,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.513244,0.005711,0.213781,0.251819,0.079795,0.000000,0.0,0.017594,0.007208,0.001288,...,74.221670,0.347202,0.361672,1.0,1.0,0.0,1.0,1.0,1.0,1.0
4,0.390319,0.009454,0.272884,0.373487,0.000000,0.000000,0.0,0.000000,0.000000,0.000196,...,61.546536,0.437852,0.673196,1.0,0.0,0.0,1.0,1.0,1.0,1.0


In [2]:
df1.shape

(98, 69)

In [3]:
missing=pd.DataFrame(df1.isna().any(),columns=["missing?"])
missing[missing["missing?"]==True]

,missing?


No missing value.

In [4]:
dtypes=pd.DataFrame(df1.dtypes,columns=["dtypes"])
dtypes[dtypes["dtypes"]=="object"]

,dtypes


All numeric values.

In [5]:
X1=df1.drop(["experts::0","experts::1","experts::2","experts::3","experts::4","experts::5","consensus"],axis=1)
X1.head()

,cervix_area,os_area,walls_area,speculum_area,artifacts_area,cervix_artifacts_area,os_artifacts_area,walls_artifacts_area,speculum_artifacts_area,cervix_specularities_area,...,fit_cervix_hull_total,fit_cervix_bbox_rate,fit_cervix_bbox_total,fit_circle_rate,fit_circle_total,fit_ellipse_rate,fit_ellipse_total,fit_ellipse_goodness,dist_to_center_cervix,dist_to_center_os
0,0.344647,0.003080,0.047522,0.288216,0.178585,0.016564,0.0,0.043500,0.010149,0.000133,...,0.373371,0.844454,0.408130,0.603399,0.571175,0.962995,0.357890,85.474311,0.265933,0.346294
1,0.165329,0.000000,0.048236,0.504736,0.502783,0.007012,0.0,0.097405,0.973837,0.004055,...,0.194308,0.646645,0.255673,0.497315,0.332444,0.894625,0.184803,124.794129,1.000000,0.283059
2,0.457010,0.001681,0.242888,0.212859,0.000000,0.000000,0.0,0.000000,0.000000,0.001756,...,0.497554,0.747443,0.611432,0.633925,0.720923,0.920287,0.496596,94.948697,0.518740,0.419375
3,0.513244,0.005711,0.213781,0.251819,0.079795,0.000000,0.0,0.017594,0.007208,0.001288,...,0.539286,0.855409,0.599998,0.618140,0.830304,0.964611,0.532073,74.221670,0.347202,0.361672
4,0.390319,0.009454,0.272884,0.373487,0.000000,0.000000,0.0,0.000000,0.000000,0.000196,...,0.408286,0.882990,0.442043,0.623938,0.625574,0.957604,0.407600,61.546536,0.437852,0.673196


In [6]:
y1=df1[["consensus"]]
y1.value_counts()

consensus
1.0          67
0.0          31
dtype: int64

In [7]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(X1, np.asarray(y1).reshape(y1.shape[0],))
features = fit.transform(X1)
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
55,18.819881
41,15.472032
40,13.208799
51,11.599963
22,10.695596
...,...
37,0.049601
6,0.027238
5,0.004102
56,0.001418


In [8]:
X1_scores=scores[scores["scores"]>10]
len(X1_scores)

10

In [9]:
X1p=X1.iloc[:,X1_scores.index]
X1p.head()

,fit_circle_rate,hsv_cervix_s_mean,hsv_cervix_h_std,fit_cervix_hull_rate,rgb_cervix_g_mean_plus_std,hsv_cervix_v_std,rgb_cervix_g_std,fit_ellipse_rate,rgb_total_g_mean,hsv_total_v_mean
0,0.603399,167.952780,2.991944,0.923067,148.653865,38.733741,38.735421,0.962995,95.109755,95.123889
1,0.497315,130.260492,2.965108,0.850861,167.108482,44.743932,44.742407,0.894625,101.680459,101.725519
2,0.633925,163.576979,2.983966,0.918514,159.296876,49.753659,49.753490,0.920287,109.592342,109.597127
3,0.618140,156.242754,2.969214,0.951710,159.668211,43.582671,43.593124,0.964611,102.641859,102.648278
4,0.623938,158.160578,2.964685,0.955996,138.216879,37.163383,37.171973,0.957604,97.446185,97.457304


In [10]:
from sklearn.preprocessing import StandardScaler
scaler1=StandardScaler()
X1p=pd.DataFrame(scaler1.fit_transform(X1p),columns=X1p.columns)
X1p.head()

,fit_circle_rate,hsv_cervix_s_mean,hsv_cervix_h_std,fit_cervix_hull_rate,rgb_cervix_g_mean_plus_std,hsv_cervix_v_std,rgb_cervix_g_std,fit_ellipse_rate,rgb_total_g_mean,hsv_total_v_mean
0,0.106413,-0.133949,0.209255,0.055653,-0.056419,-0.089915,-0.092996,0.117650,-0.031197,-0.033523
1,-0.635816,-0.894061,0.164500,-0.317217,0.196370,0.242962,0.239641,-0.225384,0.131989,0.130497
2,0.319988,-0.222192,0.195950,0.032145,0.089368,0.520428,0.517130,-0.096632,0.328483,0.326070
3,0.209548,-0.370096,0.171348,0.203567,0.094454,0.178645,0.176000,0.125755,0.155866,0.153423
4,0.250119,-0.331421,0.163795,0.225698,-0.199383,-0.176890,-0.179572,0.090600,0.026829,0.024452


In [11]:
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import LeaveOneOut

In [12]:
from sklearn import svm
model1=svm.SVC(C=3,kernel="poly",random_state=0)
scores=cross_val_score(model1,X1p,np.asarray(y1).reshape(y1.shape[0],),cv=3)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.7651515151515151 +/- 0.016365506814161247


In [13]:
y1_pred = cross_val_predict(model1, X1p, np.asarray(y1).reshape(y1.shape[0],), cv=3)
print("AUC: ",roc_auc_score(y1,y1_pred))

AUC:  0.6463649494463168


In [14]:
print("Confusion matrix: \n",confusion_matrix(y1,y1_pred))

Confusion matrix: 
 [[10 21]
 [ 2 65]]


In [15]:
print("Specificity: ",10/31)
print("Sensitivity: ",65/67)

Specificity:  0.3225806451612903
Sensitivity:  0.9701492537313433


In [16]:
from imblearn.ensemble import BalancedBaggingClassifier
model2=BalancedBaggingClassifier(base_estimator=svm.SVC(C=7,gamma="auto",random_state=0),random_state=0)
scores=cross_val_score(model2,X1p,np.asarray(y1).reshape(y1.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.80625 +/- 0.032497328950067406


In [17]:
y1_pred = cross_val_predict(model2, X1p, np.asarray(y1).reshape(y1.shape[0],), cv=4)
print("AUC: ",roc_auc_score(y1,y1_pred))

AUC:  0.7802118440057776


In [18]:
print("Confusion matrix: \n",confusion_matrix(y1,y1_pred))

Confusion matrix: 
 [[22  9]
 [10 57]]


In [19]:
print("Specificity: ",22/31)
print("Sensitivity: ",57/67)

Specificity:  0.7096774193548387
Sensitivity:  0.8507462686567164


In [20]:
from imblearn.ensemble import BalancedRandomForestClassifier
model3=BalancedRandomForestClassifier(criterion="entropy",sampling_strategy="not majority",
                                      random_state=0)
scores=cross_val_score(model3,X1p,np.asarray(y1).reshape(y1.shape[0],),cv=5)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8052631578947368 +/- 0.04233490010987412


In [21]:
y1_pred = cross_val_predict(model3, X1p, np.asarray(y1).reshape(y1.shape[0],), cv=5)
print("AUC: ",roc_auc_score(y1,y1_pred))

AUC:  0.7455464612421762


In [22]:
print("Confusion matrix: \n",confusion_matrix(y1,y1_pred))

Confusion matrix: 
 [[18 13]
 [ 6 61]]


In [23]:
from sklearn.ensemble import RandomForestClassifier
model4=RandomForestClassifier(random_state=0)
score4=cross_val_score(model4,X1p,np.asarray(y1).reshape(y1.shape[0],),cv=5)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8052631578947368 +/- 0.04233490010987412


In [24]:
y1_pred = cross_val_predict(model4, X1p, np.asarray(y1).reshape(y1.shape[0],), cv=5)
print("AUC: ",roc_auc_score(y1,y1_pred))

AUC:  0.7144920558497834


In [25]:
print("Confusion matrix: \n",confusion_matrix(y1,y1_pred))

Confusion matrix: 
 [[17 14]
 [ 8 59]]


#### Although BalancedRandomForestClassifier and RandomForestClassifier delivered the same accuracy with 5-fold cross validation, BalancedRandomForestClassifier has a higher AUC and is therefore a more promising model for the "green" dataset. This is not surprising. Class distribution is out of balanced. BalancedRandomForestClassifier has built-in functions to handle imbalanced class distribution whereas RandomForestClassifier does not.

#### The most promising model for the "green" dataset is BalancedBaggingClassifier with SVM as the base estimator.

### II. The Hinselmann dataset

In [26]:
url2="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_eb1b34f510364cad888d953cc8f19045.csv"
df2=pd.read_csv(url2)
df2.head()

,cervix_area,os_area,walls_area,speculum_area,artifacts_area,cervix_artifacts_area,os_artifacts_area,walls_artifacts_area,speculum_artifacts_area,cervix_specularities_area,...,fit_ellipse_goodness,dist_to_center_cervix,dist_to_center_os,experts::0,experts::1,experts::2,experts::3,experts::4,experts::5,consensus
0,0.334652,0.002727,0.054515,0.265214,0.165955,0.006267,0.0,0.034324,0.009077,0.001381,...,100.911343,0.256455,0.404257,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.441332,0.002393,0.157184,0.204246,0.052486,0.000000,0.0,0.315968,0.003177,0.010093,...,65.515920,0.566759,0.491791,1.0,1.0,1.0,0.0,1.0,1.0,1.0
2,0.486323,0.017873,0.211711,0.245662,0.039998,0.000000,0.0,0.003168,0.001563,0.003664,...,84.437574,0.466575,0.491928,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.383839,0.006636,0.158500,0.377851,0.046453,0.000000,0.0,0.010741,0.013657,0.001535,...,66.491785,0.371283,0.632375,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.613026,0.007757,0.091303,0.235390,0.000000,0.000000,0.0,0.000000,0.000000,0.001109,...,88.033412,0.593152,0.364539,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [27]:
df2.shape

(97, 69)

In [28]:
missing=pd.DataFrame(df2.isna().any(),columns=["missing?"])
missing[missing["missing?"]==True]

,missing?


No missing value.

In [29]:
dtypes=pd.DataFrame(df2.dtypes,columns=["dtypes"])
dtypes[dtypes["dtypes"]=="object"]

,dtypes


All numeric values.

In [30]:
X2=df2.drop(["experts::0","experts::1","experts::2","experts::3","experts::4","experts::5","consensus"],axis=1)
X2.head()

,cervix_area,os_area,walls_area,speculum_area,artifacts_area,cervix_artifacts_area,os_artifacts_area,walls_artifacts_area,speculum_artifacts_area,cervix_specularities_area,...,fit_cervix_hull_total,fit_cervix_bbox_rate,fit_cervix_bbox_total,fit_circle_rate,fit_circle_total,fit_ellipse_rate,fit_ellipse_total,fit_ellipse_goodness,dist_to_center_cervix,dist_to_center_os
0,0.334652,0.002727,0.054515,0.265214,0.165955,0.006267,0.0,0.034324,0.009077,0.001381,...,0.374878,0.811251,0.412513,0.580851,0.576141,0.947223,0.353298,100.911343,0.256455,0.404257
1,0.441332,0.002393,0.157184,0.204246,0.052486,0.000000,0.0,0.315968,0.003177,0.010093,...,0.453333,0.876909,0.503282,0.658180,0.670534,0.966245,0.456750,65.515920,0.566759,0.491791
2,0.486323,0.017873,0.211711,0.245662,0.039998,0.000000,0.0,0.003168,0.001563,0.003664,...,0.513859,0.875962,0.555186,0.604088,0.805053,0.964174,0.504393,84.437574,0.466575,0.491928
3,0.383839,0.006636,0.158500,0.377851,0.046453,0.000000,0.0,0.010741,0.013657,0.001535,...,0.407426,0.879969,0.436196,0.588115,0.652659,0.961196,0.399334,66.491785,0.371283,0.632375
4,0.613026,0.007757,0.091303,0.235390,0.000000,0.000000,0.0,0.000000,0.000000,0.001109,...,0.632647,0.769578,0.796575,0.651749,0.940586,0.988365,0.620243,88.033412,0.593152,0.364539


In [31]:
y2=df2[["consensus"]]
y2.value_counts()

consensus
1.0          82
0.0          15
dtype: int64

In [32]:
fit = test.fit(X2, np.asarray(y2).reshape(y2.shape[0],))
features = fit.transform(X2)
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
60,15.225542
42,4.996002
2,4.221798
59,3.495606
16,3.091918
...,...
6,0.001812
53,0.000339
50,0.000284
21,0.000043


In [33]:
X2_scores=scores[scores["scores"]>3]
len(X2_scores)

5

In [34]:
X2p=X2.iloc[:,X2_scores.index]
X2p.head()

,dist_to_center_cervix,hsv_cervix_s_std,walls_area,fit_ellipse_goodness,rgb_cervix_r_std
0,0.256455,25.751500,0.054515,100.911343,46.272693
1,0.566759,24.989313,0.157184,65.515920,59.336903
2,0.466575,23.277555,0.211711,84.437574,46.395580
3,0.371283,23.235599,0.158500,66.491785,47.230657
4,0.593152,26.866524,0.091303,88.033412,43.919992


In [35]:
scaler2=StandardScaler()
X2p=pd.DataFrame(scaler2.fit_transform(X2p),columns=X2p.columns)
X2p.head()

,dist_to_center_cervix,hsv_cervix_s_std,walls_area,fit_ellipse_goodness,rgb_cervix_r_std
0,-0.970480,-0.162124,-0.613976,-0.368077,0.326954
1,0.352130,-0.239019,-0.043744,-0.784388,1.315352
2,-0.074884,-0.411713,0.259102,-0.561837,0.336251
3,-0.481047,-0.415946,-0.036439,-0.772910,0.399430
4,0.464628,-0.049633,-0.409651,-0.519543,0.148955


In [36]:
model5=BalancedBaggingClassifier(base_estimator=svm.SVC(C=3,kernel="poly",degree=5,random_state=0),
                                 random_state=0)
scores=cross_val_score(model5,X2p,np.asarray(y2).reshape(y2.shape[0],),cv=3)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8866792929292929 +/- 0.01392471275259645


In [37]:
y2_pred = cross_val_predict(model5, X2p, np.asarray(y2).reshape(y2.shape[0],), cv=3)
print("AUC: ",roc_auc_score(y2,y2_pred))

AUC:  0.6333333333333333


In [38]:
print("Confusion matrix: \n",confusion_matrix(y2,y2_pred))

Confusion matrix: 
 [[ 4 11]
 [ 0 82]]


In [39]:
model6=svm.SVC(C=3,kernel="poly",degree=5,random_state=0)
scores=cross_val_score(model6,X2p,np.asarray(y2).reshape(y2.shape[0],),cv=3)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8661616161616162 +/- 0.012499362293701586


In [40]:
y2_pred = cross_val_predict(model6, X2p, np.asarray(y2).reshape(y2.shape[0],), cv=3)
print("AUC: ",roc_auc_score(y2,y2_pred))

AUC:  0.6211382113821138


In [41]:
print("Confusion matrix: \n",confusion_matrix(y2,y2_pred))

Confusion matrix: 
 [[ 4 11]
 [ 2 80]]


In [42]:
model7=RandomForestClassifier(criterion="entropy",min_samples_leaf=2,random_state=0)
scores=cross_val_score(model7,X2p,np.asarray(y2).reshape(y2.shape[0],),cv=3)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8554292929292929 +/- 0.03039495736669374


In [43]:
y2_pred = cross_val_predict(model7, X2p, np.asarray(y2).reshape(y2.shape[0],), cv=3)
print("AUC: ",roc_auc_score(y2,y2_pred))

AUC:  0.5605691056910569


In [44]:
print("Confusion matrix: \n",confusion_matrix(y2,y2_pred))

Confusion matrix: 
 [[ 2 13]
 [ 1 81]]


In [45]:
model8=BalancedRandomForestClassifier(criterion="entropy",sampling_strategy="not majority",min_samples_leaf=2,
                                      random_state=0)
scores=cross_val_score(model8,X2p,np.asarray(y2).reshape(y2.shape[0],),cv=3)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8762626262626263 +/- 0.025577922449347087


In [46]:
y2_pred = cross_val_predict(model8, X2p, np.asarray(y2).reshape(y2.shape[0],), cv=3)
print("AUC: ",roc_auc_score(y2,y2_pred))

AUC:  0.6272357723577235


In [47]:
print("Confusion matrix: \n",confusion_matrix(y2,y2_pred))

Confusion matrix: 
 [[ 4 11]
 [ 1 81]]


#### For the "Hinselmann" dataset, the classifiers from the "imblearn.ensemble" library outperformed the their counterparts from the sklearn.ensemble" library again. 

#### For this dataset, BalancedBaggingClassifier with SVM as the base estimator is still the most promising algorithm.

### III. The Schiller dataset

In [48]:
url3="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_8b87184c07d94d878002116d851df19f.csv"
df3=pd.read_csv(url3)
df3.head()

,cervix_area,os_area,walls_area,speculum_area,artifacts_area,cervix_artifacts_area,os_artifacts_area,walls_artifacts_area,speculum_artifacts_area,cervix_specularities_area,...,fit_ellipse_goodness,dist_to_center_cervix,dist_to_center_os,experts::0,experts::1,experts::2,experts::3,experts::4,experts::5,consensus
0,0.317899,0.008975,0.142670,0.264625,0.051382,0.022907,0.0,0.000291,0.016850,0.003351,...,178.019319,0.327891,0.486702,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.551028,0.007583,0.096114,0.241423,0.056713,0.012370,0.0,0.132463,0.039886,0.003695,...,86.659527,0.570223,0.532907,1.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.570135,0.008553,0.212171,0.262682,0.052377,0.015785,0.0,0.021698,0.010502,0.002046,...,101.940392,0.517132,0.554941,1.0,1.0,1.0,1.0,0.0,0.0,1.0
3,0.456403,0.006923,0.118763,0.372608,0.047079,0.010110,0.0,0.009945,0.029686,0.005111,...,75.267157,0.311316,0.596119,0.0,1.0,0.0,1.0,1.0,0.0,1.0
4,0.560898,0.009135,0.147885,0.294300,0.005061,0.009024,0.0,0.000000,0.000000,0.003084,...,101.926838,0.618853,0.404181,0.0,1.0,0.0,1.0,1.0,0.0,1.0


In [49]:
df3.shape

(92, 69)

In [50]:
missing=pd.DataFrame(df3.isna().any(),columns=["missing?"])
missing[missing["missing?"]==True]

,missing?


No missing value.

In [51]:
dtypes=pd.DataFrame(df3.dtypes,columns=["dtypes"])
dtypes[dtypes["dtypes"]=="object"]

,dtypes


All numeric values.

In [52]:
X3=df3.drop(["experts::0","experts::1","experts::2","experts::3","experts::4","experts::5","consensus"],axis=1)
X3.head()

,cervix_area,os_area,walls_area,speculum_area,artifacts_area,cervix_artifacts_area,os_artifacts_area,walls_artifacts_area,speculum_artifacts_area,cervix_specularities_area,...,fit_cervix_hull_total,fit_cervix_bbox_rate,fit_cervix_bbox_total,fit_circle_rate,fit_circle_total,fit_ellipse_rate,fit_ellipse_total,fit_ellipse_goodness,dist_to_center_cervix,dist_to_center_os
0,0.317899,0.008975,0.142670,0.264625,0.051382,0.022907,0.0,0.000291,0.016850,0.003351,...,0.337419,0.832180,0.382007,0.611869,0.519554,0.964937,0.329451,178.019319,0.327891,0.486702
1,0.551028,0.007583,0.096114,0.241423,0.056713,0.012370,0.0,0.132463,0.039886,0.003695,...,0.613848,0.823647,0.669010,0.640770,0.859946,0.970144,0.567986,86.659527,0.570223,0.532907
2,0.570135,0.008553,0.212171,0.262682,0.052377,0.015785,0.0,0.021698,0.010502,0.002046,...,0.633357,0.781640,0.729410,0.577287,0.987611,0.944082,0.603904,101.940392,0.517132,0.554941
3,0.456403,0.006923,0.118763,0.372608,0.047079,0.010110,0.0,0.009945,0.029686,0.005111,...,0.489925,0.855169,0.533699,0.616696,0.740077,0.965694,0.472616,75.267157,0.311316,0.596119
4,0.560898,0.009135,0.147885,0.294300,0.005061,0.009024,0.0,0.000000,0.000000,0.003084,...,0.587964,0.776697,0.722157,0.646024,0.868231,0.978936,0.572967,101.926838,0.618853,0.404181


In [53]:
y3=df3[["consensus"]]
y3.value_counts()

consensus
1.0          67
0.0          25
dtype: int64

In [54]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(X3, np.asarray(y3).reshape(y3.shape[0],))
features = fit.transform(X3)
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
1,8.399068e+00
44,6.891207e+00
60,6.858578e+00
26,6.841748e+00
24,6.675419e+00
...,...
46,1.599493e-02
17,1.173514e-02
42,3.480840e-03
56,2.120290e-03


In [55]:
X3_scores=scores[scores["scores"]>6.5]
len(X3_scores)

5

In [56]:
X3p=X3.iloc[:,X3_scores.index]
X3p.head()

,os_area,hsv_cervix_v_std,dist_to_center_cervix,rgb_cervix_b_mean_plus_std,rgb_cervix_b_std
0,0.008975,26.895496,0.327891,112.498460,27.884325
1,0.007583,32.303187,0.570223,91.692691,32.791010
2,0.008553,21.569742,0.517132,83.865919,20.957295
3,0.006923,39.869724,0.311316,129.333598,41.534182
4,0.009135,28.872137,0.618853,134.437099,31.661283


In [57]:
scaler3=StandardScaler()
X3p=pd.DataFrame(scaler3.fit_transform(X3p),columns=X3p.columns)
X3p.head()

,os_area,hsv_cervix_v_std,dist_to_center_cervix,rgb_cervix_b_mean_plus_std,rgb_cervix_b_std
0,0.423025,-0.998837,-0.682031,-0.314725,-0.953593
1,0.150390,-0.732826,0.262691,-0.658613,-0.714266
2,0.340423,-1.260817,0.055718,-0.787977,-1.291463
3,0.021011,-0.360619,-0.746648,-0.036465,-0.287812
4,0.454365,-0.901604,0.452272,0.047888,-0.769369


In [58]:
model9=svm.SVC(C=3,random_state=0)
scores=cross_val_score(model9,X3p,np.asarray(y3).reshape(y3.shape[0],),cv=3)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8154121863799283 +/- 0.012672164537393366


In [59]:
y3_pred = cross_val_predict(model9, X3p, np.asarray(y3).reshape(y3.shape[0],), cv=3)
print("AUC: ",roc_auc_score(y3,y3_pred))

AUC:  0.7101492537313433


In [60]:
print("Confusion matrix: \n",confusion_matrix(y3,y3_pred))

Confusion matrix: 
 [[12 13]
 [ 4 63]]


In [61]:
model10=BalancedBaggingClassifier(base_estimator=svm.SVC(C=2,kernel="poly",random_state=0),sampling_strategy="majority",
                                  random_state=0)
scores=cross_val_score(model10,X3p,np.asarray(y3).reshape(y3.shape[0],),cv=5)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8040935672514621 +/- 0.027923593886113023


In [62]:
y3_pred = cross_val_predict(model10, X3p, np.asarray(y3).reshape(y3.shape[0],), cv=5)
print("AUC: ",roc_auc_score(y3,y3_pred))

AUC:  0.6901492537313433


In [63]:
print("Confusion matrix: \n",confusion_matrix(y3,y3_pred))

Confusion matrix: 
 [[11 14]
 [ 4 63]]


In [64]:
model11=RandomForestClassifier(random_state=0)
scores=cross_val_score(model11,X3p,np.asarray(y3).reshape(y3.shape[0],),cv=3)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8154121863799283 +/- 0.02922850614749123


In [65]:
y3_pred = cross_val_predict(model11, X3p, np.asarray(y3).reshape(y3.shape[0],), cv=3)
print("AUC: ",roc_auc_score(y3,y3_pred))

AUC:  0.7101492537313433


In [66]:
print("Confusion matrix: \n",confusion_matrix(y3,y3_pred))

Confusion matrix: 
 [[12 13]
 [ 4 63]]


In [67]:
model12=BalancedRandomForestClassifier(sampling_strategy="not majority",random_state=0)
scores=cross_val_score(model12,X3p,np.asarray(y3).reshape(y3.shape[0],),cv=3)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.7935483870967742 +/- 0.013937076770769592


In [68]:
y3_pred = cross_val_predict(model12, X3p, np.asarray(y3).reshape(y3.shape[0],), cv=3)
print("AUC: ",roc_auc_score(y3,y3_pred))

AUC:  0.6826865671641791


In [69]:
print("Confusion matrix: \n",confusion_matrix(y3,y3_pred))

Confusion matrix: 
 [[11 14]
 [ 5 62]]


#### For the "Schiller" dataset, the classifiers from the "imblearn.ensemble" library slightly underperformed when compared to their counterparts from the "sklearn" library. SVM was ore stable and more accurate by itself than when it was used as a base estimator for BalancedBaggingClassifier. RandomForestClassifier outperformed BalancedRandomForestClassifier.

#### Fort this dataset, the most accurate and most stable model is SVM.

### IV. Summary

#### Each dataset has 62 predictive features and 7 target variales that are the physician's assessments of the quality of the digital olposcopies. The goal here is to predict the 7th target variable, the "concensus" assessment.

|Dataset|Model|Model NO.|Number of features used|Cross-validation fold|Average cross-validation accuracy|AUC|
|-------|-----|---------|-----------------------|---------------------|---------------------------------|---|
|Green|SVM with "poly" kernel|1|10|3|76.52% +/- 1.64%|0.65|
|Green|BalancedBaggingClassifier with SVM as base estimator|2|10|4|80.6% +/- 3.2%|0.78|
|Green|BalancedRandomForestClassifier|3|10|5|80.5% +/- 4.2%|0.75|
|Green|RandomForestClassifier|4|10|5|80.5% +/- 4.2%|0.714|
|Hinselmann|BalancedBaggingClassifier with SVM as base estimator|5|5|3|88.7% +/- 1.4%|0.63|
|Hinselmann|SVM with "poly" kernel|6|5|3|86.6 +/- 1.2%|0.62|
|Hinselmann|RandomForestClassifier|7|5|3|85.5% +/- 3%|0.56|
|Hinselmann|BalancedRandomForestClassifier|8|5|3|87.6% +/- 2.6%|0.63|
|Schiller|SVM with "rbf" kernel|9|5|3|81.5% +/- 1.3%|0.71|
|Schiller|BalancedBaggingClassifier with SVM as the base estimator|10|5|5|80.4% +/- 2.4%|0.69|
|Schiller|RandomForestClassifier|11|5|3|81.5% +/- 2.9%|0.71|
|Schiller|BalancedRandomForestClassifier|12|5|3|79% +/- 1.4%|0.68|